In [66]:
import numpy as np
import pandas as pd
from PIL import Image
import os
from skimage import io
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier
import seaborn as sns

In [18]:
# train_labels = pd.read_csv("train.csv", index_col="image_id")
# test_labels = pd.read_csv("test.csv", index_col="image_id")
# validation_labels = pd.read_csv("validation.csv", index_col="image_id")

train_labels = pd.read_csv("train.csv")
test_labels = pd.read_csv("test.csv")
validation_labels = pd.read_csv("validation.csv")

In [41]:
def show_image(image):
    io.imshow(image.astype(np.uint8))
    io.show()

In [50]:
def normalize_data(data, type=None):
    if type == None:
        return (data)
    elif type == 'l2':
        return normalize(data, norm='l2')
    elif type == 'l1':
        return normalize(data, norm='l1')

In [4]:
def calculeaza_std_si_mean(folder, label_file):
    pathtodir = os.getcwd()
    pathtodir = os.path.join(pathtodir, folder)
    print(pathtodir)

    aux_imagini = []
    cnt = 0
    for nume_imagine in tqdm(label_file.loc[:, "image_id"]):
        image_name = nume_imagine + ".png"
        pathToImage = os.path.join(pathtodir, image_name)
        imagine = np.array(Image.open(pathToImage))
        imagine = np.ravel(imagine.copy())
        if(imagine.shape == (6400, )):
            # if folder == "train":#in viitor va trb sa sterg imaginile alb/negru
            #     continue
            imagine = np.concatenate((imagine, imagine, imagine))
        #imagine = np.digitize(imagine, intervale)
        #imagine = (imagine - min(imagine))/(max(imagine) - min(imagine))
        aux_imagini.append(imagine)
    imagini = np.array(aux_imagini)
    return (np.mean(imagini), np.std(imagini))
    

In [42]:
def load_images_from_dir(folder, label_file):
    pathtodir = os.getcwd()
    pathtodir = os.path.join(pathtodir, folder)
    print(pathtodir)

    aux_imagini = []
    intervale = np.linspace(start=0, stop=1.1, num=25)
    cnt = 0
    for nume_imagine in tqdm(label_file.loc[:, "image_id"]):
        image_name = nume_imagine + ".png"
        pathToImage = os.path.join(pathtodir, image_name)
        imagine = np.array(Image.open(pathToImage))
        imagine = np.ravel(imagine.copy())
        if(imagine.shape == (6400, )):
            # if folder == "train":#in viitor va trb sa sterg imaginile alb/negru
            #     continue
            imagine = np.concatenate((imagine, imagine, imagine))
        #img = imagine.copy()
        imagine = imagine.astype(np.uint8)
        aux_imagini.append(imagine)
    imagini = np.array(aux_imagini)
    return imagini
    

In [9]:
mean, std = calculeaza_std_si_mean("train", train_labels)

NameError: name 'calculeaza_std_si_mean' is not defined

In [69]:
train_images = load_images_from_dir("train", train_labels)
test_images = load_images_from_dir("test", test_labels)
validation_images = load_images_from_dir("validation", validation_labels)

C:\Users\Fabi\Desktop\ml\kaggle\Realistic Image Classification @ UnivBuc\train


100%|██████████████████████████████████████████████████████████████████████████| 10500/10500 [00:04<00:00, 2301.41it/s]


C:\Users\Fabi\Desktop\ml\kaggle\Realistic Image Classification @ UnivBuc\test


100%|████████████████████████████████████████████████████████████████████████████| 4500/4500 [00:01<00:00, 2263.58it/s]


C:\Users\Fabi\Desktop\ml\kaggle\Realistic Image Classification @ UnivBuc\validation


100%|████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:01<00:00, 2291.82it/s]


In [ ]:
train_images = normalize_data(train_images,type='l1')
test_images = normalize_data(test_images, type='l1')
validation_images = normalize_data(validation_images, type='l1')

In [ ]:
print(train_images[2].shape)

In [ ]:
show_image(np.reshape(train_images[120], (80, 80, 3))) #279 e interesant
print(max(validation_images[279]))

In [ ]:
#incercam un model simplu doar asa ca exemplu
model = SVC(C=2.5)
model.fit(train_images, train_labels.iloc[:, 1])
predictions = model.predict(validation_images)
scor = accuracy_score(validation_labels.iloc[:, 1], predictions)
scor

In [60]:
model1 = MLPClassifier(hidden_layer_sizes = 200)
model1.fit(train_images, train_labels.iloc[:, 1])
predictions = model.predict(validation_images)
scor = accuracy_score(validation_labels.iloc[:, 1], predictions)
scor

C:\Users\Fabi\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.516

In [65]:
model2 = XGBClassifier()
model2.fit(train_images, train_labels.iloc[:, 1])
predictions = model2.predict(validation_images)
scor = accuracy_score(validation_labels.iloc[:, 1], predictions)
scor

NameError: name 'XGBClassifier' is not defined

In [35]:
predictii_test = model.predict(test_images)
raspuns = pd.Series(data=predictii_test, index=test_labels.iloc[:, 0], name="label")
raspuns.to_csv("raspuns.csv")